In [179]:

import random
import numpy as np 
import os
import torch
import pandas as pd

def seed_everything(seed: int = 14):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    print(f"Seed set as {seed}")


seed_everything(14)

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

def swap_word(new_words):
    random.seed(14)
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random.seed(4111)
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 2:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def EDA(sentence, alpha_rs=0.1 , num_aug=4):
    words = sentence.split(' ')
    words = [word for word in words if word is not ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4) + 1

    n_rs = max(1, int(alpha_rs*num_words))

        # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    # random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = list(set(augmented_sentences[:num_aug]))[0]

    return augmented_sentences

Seed set as 14


In [180]:
# .. 상위 폴더
train_pd = pd.read_csv('../../../../data/train.csv')
train_pd

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [181]:
aug_pd = train_pd.copy(deep=True)
aug_pd['sentence_1']=aug_pd['sentence_1'].apply(lambda x: EDA(x)) 
aug_pd['sentence_2']=aug_pd['sentence_2'].apply(lambda x: EDA(x))

In [182]:
aug_pd

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 다르네요~ 있고 여느 한국영화 쓰레기들하고는 차원이 반전도,"있고재미도있네요. 있고,사랑도 반전도",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,뜹니다;; 제가 접근권한이 없다고 앗,"합니다. 액세스 권한이 없다고 오,",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,변경해주세요. 주택청약조건,변경해주세요. 무주택기준 주택청약,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,반가웠습니다. 처음 대면으로 만나 입사후,반가웠습니다. 보다가 리얼로 만나니 정말 화상으로만,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,하네요!! 뿌듯뿌듯,뿌뿌뿌~!~! 실제로 한번 뵈어요 꼬옥,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,장치를 교원이 보호받을 수 있는 교원능력개발평가에서 마련해야합니다,본인이 주세요 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 납부한,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,원합니드 폐지를 여성가족부의,합니다. 폐지를 청원 여성가족부,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,줄여주세요 월급좀 국회의원들,마세요 봉급좀 줄이지좀 공무원,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 해요!! 점심은 다음에 다시 츄라이 하기로 못한,먹기로 못먹은 밥은 꼭 담에 오늘 하고요!!,3.2,1.0


In [183]:
train_pd=pd.concat([train_pd,aug_pd]).reset_index()

In [184]:
train_pd

,index,id,source,sentence_1,sentence_2,label,binary-label
0,0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...,...
18643,9319,boostcamp-sts-v1-train-9319,petition-sampled,장치를 교원이 보호받을 수 있는 교원능력개발평가에서 마련해야합니다,본인이 주세요 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 납부한,0.2,0.0
18644,9320,boostcamp-sts-v1-train-9320,petition-sampled,원합니드 폐지를 여성가족부의,합니다. 폐지를 청원 여성가족부,4.2,1.0
18645,9321,boostcamp-sts-v1-train-9321,petition-sampled,줄여주세요 월급좀 국회의원들,마세요 봉급좀 줄이지좀 공무원,0.6,0.0
18646,9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 해요!! 점심은 다음에 다시 츄라이 하기로 못한,먹기로 못먹은 밥은 꼭 담에 오늘 하고요!!,3.2,1.0
